In [27]:
import sys
import os

os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-1.8.0-openjdk-amd64'
os.environ['PYSPARK_PYTHON'] = '/home/aadi/miniconda3/envs/pyspark_env/bin/python' 
os.environ['PYSPARK_DRIVER_PYTHON'] = '/home/aadi/miniconda3/envs/pyspark_env/bin/python' 

In [28]:
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors

sc = SparkSession.Builder().appName('vif').getOrCreate()
spark = SparkSession.Builder().appName('vif').getOrCreate()

In [30]:
import pandas as pd
import numpy as np


N = 5000
data = pd.DataFrame({
    'id': ['a'] * N,
    'num1': np.random.normal(1000, 100, size=N), 
    'num2': np.random.randint(low=-10, high=10, size=N),
    'num3': np.random.randint(low=-10, high=10, size=N),
    'num4': np.random.randint(low=-10, high=10, size=N),
    'num5': np.random.randint(low=-10, high=10, size=N),
    'target': np.random.choice([0, 1],p=[0.9, 0.1],  size=N)
})

data = data.assign(num3=data.num2 * 3)
data = data.assign(num4=data.num3 + data.num5)

dd = sc.createDataFrame(data)
dd.show()

/home/aadi/miniconda3/envs/pyspark_env/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/aadi/miniconda3/envs/pyspark_env/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+---+------------------+----+----+----+----+------+
| id|              num1|num2|num3|num4|num5|target|
+---+------------------+----+----+----+----+------+
|  a| 950.8331744353978|   1|   3|  -6|  -9|     0|
|  a|1040.5260121163003|   9|  27|  31|   4|     0|
|  a|1170.6939202412382|   3|   9|   3|  -6|     1|
|  a| 949.8853098157743|  -1|  -3|  -2|   1|     0|
|  a|1106.6040033862125|   2|   6|  -2|  -8|     0|
|  a|1032.0706922290929|   6|  18|  11|  -7|     0|
|  a|1031.5072744790161|   3|   9|   3|  -6|     0|
|  a|  934.001633318011|   9|  27|  34|   7|     0|
|  a|1016.3160198901106|   6|  18|  26|   8|     0|
|  a| 997.3965983827237|   4|  12|   7|  -5|     0|
|  a|  1110.45876864105|   5|  15|  20|   5|     0|
|  a| 862.9216752584423|   2|   6|  12|   6|     0|
|  a|1088.0622867177278|  -8| -24| -20|   4|     0|
|  a|1014.6049153786838|   5|  15|   9|  -6|     0|
|  a|1031.3089475617617|  -7| -21| -18|   3|     0|
|  a|1214.5055095248392|   6|  18|  13|  -5|     1|
|  a| 862.85

# Not From Github

In [24]:
import sklearn.feature_selection as fs
from statsmodels.stats.outliers_influence import variance_inflation_factor
# [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
max_iter = 100
vif_threshold = 3
exclude_cols = []
label_column = ['target']

# calculate VIF
def calculate_vif(data):
    vif_vals = [variance_inflation_factor(data.values, i) for i in range(data.drop(exclude_cols + label_column, axis=1).shape[1])]
    vif = pd.Series(index=data.drop(exclude_cols + label_column, axis=1).columns, data=vif_vals)
    max_vif = max(vif_vals)
    vif = vif.sort_values(ascending=False)

    return max_vif, vif

max_vif, vif = calculate_vif(data)
data_small = data
_iter = 0
while max_vif > vif_threshold and _iter > max_iter:
    # this assumes the highest vif is above the threshold
    # take highest two and drop one
    var1 = vif.index[0]
    var2 = vif.index[1]
    # get chi2 for both
    (chi2_v1, chi2_v2), p_values = fs.f_classif(data[[var1, var2]], np.ravel(data[label_column].values))

    if chi2_v1 > chi2_v2:
        data_small = data_small.drop(var1, axis=1)
    else:
        data_small = data_small.drop(var2, axis=1)

    max_vif, vif = calculate_vif(data_small)

/home/aadi/miniconda3/envs/pyspark_env/lib/python3.8/site-packages/statsmodels/stats/outliers_influence.py:195: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


In [25]:
data_small

,num1,num2,num3,num4,num5,target
0,976.522775,-5,-15,-25,-10,0
1,977.059625,-7,-21,-26,-5,0
2,1029.482655,1,3,3,0,0
3,995.175678,7,21,21,0,0
4,865.051425,-10,-30,-26,4,0
...,...,...,...,...,...,...
4995,951.720835,-2,-6,-13,-7,0
4996,1000.326320,-4,-12,-20,-8,0
4997,817.735320,-4,-12,-3,9,1
4998,924.923386,-7,-21,-22,-1,0


In [12]:
from pyspark.sql.types import Row

# Importing required libraries for VIF Calculation
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.linalg import DenseVector
from pyspark.ml.linalg import Vectors
from pyspark.ml.evaluation import RegressionEvaluator, BinaryClassificationEvaluator

In [13]:
vif_threshold = 5 #Threshold for VIF

class VifFilter:
  def __init__(self,
    label_column,
    exclude_cols=None,
    vif_threshold=5,
    estimator=LogisticRegression,
    evaluator=BinaryClassificationEvaluator
  ):

    self.label_column = label_column
    self.exclude_cols = exclude_cols
    self.vif_threshold = vif_threshold
    self.estimator = estimator
    self.evaluator = evaluator

  def _reorder_label(self, data):
    '''Ensures label is LAST column'''
    columns = data.columns
    columns.remove(self.label_column)
    columns.append(self.label_column)
    return data.select(*[columns])


  def calc_vif(self, data, xvar_names, vif_max, colnum_max):
    print(f'Number columns at this level: {len(data.columns)}')
    vif_max = self.vif_threshold
    for i in range(2, len(xvar_names)):
      train_table = data.rdd.map(lambda x: [Vectors.dense(x[2:i]+x[i+1:]), x[i]]).toDF(['features', 'label'])
      lr = self.estimator(featuresCol='features', labelCol='label')
      lr_model = lr.fit(train_table)
      predictions = lr_model.transform(train_table)
      evaluator = self.evaluator(predictionCol='prediction', labelCol='label')
      r_sq = evaluator.evaluate(predictions, {evaluator.metricName: 'r2'})
      vif = 1/(1-r_sq)
      if vif_max < vif:
        vif_max = vif
        colnum_max = i

      return vif_max, colnum_max

  def remove_collinear(self, data):

    data = self._reorder_label(data)

    xvar_names = data.columns
    colnum_max = len(xvar_names)
    vif_max = self.vif_threshold + 1

    while vif_max > 5:
      vif_max, colnum_max = self.calc_vif(data, xvar_names, vif_max, colnum_max)
      if vif_max > 5:
        data = data.drop(data[colnum_max])
        xvar_names = data.columns
    else:
      return data

def vif_cal_iter(inputdata,vif_threshold):
  xvar_names = inputdata.columns
  global vif_max
  global colnum_max
  colnum_max = 10000 # Initialising with a fake value
  vif_max = vif_threshold + 1
  def vif_cal(inputdata, xvar_names, vif_max, colnum_max, vif_threshold):
    print("Dimension of table at this level")
    print("================================")
    print(inputdata.count(), len(inputdata.columns))
    print("List of X Variables")
    print("===================")
    print(xvar_names)
    vif_max = vif_threshold
    for i in range(2,len(xvar_names)):
      train_t = inputdata.rdd.map(lambda x: [Vectors.dense(x[2:i]+x[i+1:]), x[i]]).toDF(['features', 'label'])
      lr = LinearRegression(featuresCol = 'features', labelCol = 'label')
      lr_model = lr.fit(train_t)
      predictions = lr_model.transform(train_t)
      evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='label')
      r_sq=evaluator.evaluate(predictions, {evaluator.metricName: "r2"})
      vif=1/(1-r_sq)
      if vif_max < vif:
        vif_max = vif
        colnum_max = i
    return vif_max, colnum_max
  while vif_max > 5:
    vif_max, colnum_max = vif_cal(inputdata, xvar_names, vif_max, colnum_max, vif_threshold)
    if vif_max > vif_threshold:
        print("Start of If Block")
        inputdata = inputdata.drop(inputdata[colnum_max])
        xvar_names = inputdata.columns
        print("Dimension of table after this iteration")
        print("=======================================")
        print(inputdata.count(), len(inputdata.columns))
        print("List of X Variables remaining")
        print("=============================")
        print(xvar_names)
  else:
    return inputdata

In [14]:
class VifFilter: pass

def calc_vif(self, data, target_name=None):
    if target_name and target_name in data.columns:
        data = data.drop(target_name)

    vifs = pd.Series(name='vif', dtype='float64')
    col_names = list(data.columns)
    for idx, colname in enumerate(col_names):
        train_table = data.rdd.map(lambda x: [Vectors.dense(x[:idx]+x[idx+1:]), x[idx]]).toDF(['features', 'label'])
        lr = LinearRegression(featuresCol='features', labelCol='label') 
        lr_model = lr.fit(train_table)
        predictions = lr_model.transform(train_table)
        eval = RegressionEvaluator(predictionCol='prediction', labelCol='label') 
        r_sq = eval.evaluate(predictions, {eval.metricName: 'r2'})

        try:
            vif = 1/(1-r_sq)
        except ZeroDivisionError as z:
            # this means perfect correlation, set to high number
            vif = float('Inf')
        vifs[colname] = vif

    return vifs.sort_values()
VifFilter.calc_vif = calc_vif

In [15]:
vf = VifFilter()
vf.calc_vif(dd, target_name='target')

23/01/25 13:05:00 WARN Instrumentation: [784b6820] regParam is zero, which might cause numerical instability and overfitting.


23/01/25 13:05:02 WARN Instrumentation: [f14d7a41] regParam is zero, which might cause numerical instability and overfitting.
23/01/25 13:05:04 WARN Instrumentation: [2f31e634] regParam is zero, which might cause numerical instability and overfitting.
23/01/25 13:05:04 WARN Instrumentation: [b1edc3e8] regParam is zero, which might cause numerical instability and overfitting.
23/01/25 13:05:06 WARN Instrumentation: [f99e0b1e] regParam is zero, which might cause numerical instability and overfitting.


num1    1.000462
num2         inf
num3         inf
num4         inf
num5         inf
Name: vif, dtype: float64

In [16]:
from pyspark.ml.stat import ChiSquareTest

def calc_target_corr(self, data, target_name):
    if target_name not in data.columns:
        raise AttributeError(f'{target_name} not found in data')

    # reorganize column order
    col_names = list(data.columns)
    col_names.remove(target_name)
    col_names.append(target_name)

    data = data.select(*[col_names])
    train_table = data.rdd.map(lambda x: [Vectors.dense(x[:-1]), x[-1]]).toDF(['features', 'label'])
    chi2res = ChiSquareTest.test(train_table, 'features', 'label', flatten=True).toPandas()
    chi2res = chi2res.assign(featureName=col_names[:-1])[['featureName', 'pValue', 'statistic']]

    return chi2res.set_index('featureName')


VifFilter.calc_chi2_corr = calc_target_corr
vf = VifFilter()

In [18]:
chi2_corr= vf.calc_chi2_corr(dd, target_name='target')
vifs = vf.calc_vif(dd, target_name='target')

23/01/25 13:06:18 WARN Instrumentation: [964bca9b] regParam is zero, which might cause numerical instability and overfitting.
23/01/25 13:06:19 WARN Instrumentation: [1cff19bb] regParam is zero, which might cause numerical instability and overfitting.
23/01/25 13:06:19 WARN Instrumentation: [2036acc8] regParam is zero, which might cause numerical instability and overfitting.
23/01/25 13:06:20 WARN Instrumentation: [eeb38c56] regParam is zero, which might cause numerical instability and overfitting.
23/01/25 13:06:21 WARN Instrumentation: [932de673] regParam is zero, which might cause numerical instability and overfitting.


In [20]:
chi2_corr

,pValue,statistic
featureName,,
num1,0.493351,5000.000000
num2,0.516780,18.084805
num3,0.516780,18.084805
num4,0.858774,62.909037
num5,0.479279,18.653176


In [207]:
vf.data = dd 
vf.vif_threshold = 5
vf.target_name = 'target'

In [208]:
self = vf
self.vif_threshold

5

In [209]:
max_vif = float('Inf')
col_names = self.data.columns
chi2_corr = self.calc_chi2_corr(self.data, self.target_name)


data = dd #self.data

while max_vif > self.vif_threshold:
    vifs = self.calc_vif(data, target_name=self.target_name)
    vif_chi2 = (
        vifs.to_frame()
        .merge(chi2_corr, left_index=True, right_index=True)
        .sort_values(['vif', 'statistic'], ascending=True)
    )

    # drop highest column value
    col_names_reduced = vif_chi2.index[:-1]
    print(col_names)
    break

22/12/23 16:02:37 WARN Instrumentation: [6568e499] regParam is zero, which might cause numerical instability and overfitting.
22/12/23 16:02:38 WARN Instrumentation: [6568e499] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.
22/12/23 16:02:38 WARN Instrumentation: [887d8047] regParam is zero, which might cause numerical instability and overfitting.
22/12/23 16:02:39 WARN Instrumentation: [430a21e0] regParam is zero, which might cause numerical instability and overfitting.
22/12/23 16:02:40 WARN Instrumentation: [3b60b1c4] regParam is zero, which might cause numerical instability and overfitting.
22/12/23 16:02:40 WARN Instrumentation: [3b60b1c4] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.
22/12/23 16:02:40 WARN Instrumentation: [d188a241] regParam is zero, which might cause numerical instability and overfitting.
22/12/23 16:02:41 WARN Instrumentation: [d188a241] Cholesky solver failed due to s

In [83]:
class VIFFilterPandas:
    def __init__(self, 
        vif_threshold,
        target_name,
        exclude_cols
    ) -> None:
        self.vif_threshold = vif_threshold
        self.target_name = target_name
        self.exclude_cols = exclude_cols


    def remove_mulitocollinear(self, data, max_iter=100):
        max_vif, vif = self.calculate_vif(data.drop(self.exclude_cols + [self.target_name], axis=1))
        if max_vif < self.vif_threshold:
            return data

        exclude_data = data[self.exclude_cols]
        data_small = data.drop(self.exclude_cols, axis=1)
        _iter = 0

        while max_vif > self.vif_threshold and _iter < max_iter:
            # this assumes the highest vif is above the threshold
            # take highest two and drop one
            var1 = vif.index[0]
            var2 = vif.index[1]
            # get chi2 for both
            (chi2_v1, chi2_v2), p_values = fs.f_classif(data_small[[var1, var2]], np.ravel(data[self.target_name].values))

            if chi2_v1 > chi2_v2:
                data_small = data_small.drop(var1, axis=1)
            else:
                data_small = data_small.drop(var2, axis=1)


            max_vif, vif = self.calculate_vif(data_small.drop(self.target_name, axis=1))
            print(max_vif, _iter)

            _iter += 1

        keep_cols = list(set(self.exclude_cols).union(set(data_small.columns)).union([self.target_name]))
        return data[keep_cols]


    def calculate_vif(self, data):
        vif_vals = [variance_inflation_factor(data, i) for i in range(data.shape[1])]
        vif = pd.Series(index=data.columns, data=vif_vals)
        max_vif = max(vif_vals)
        vif = vif.sort_values(ascending=False)

        return max_vif, vif

    

In [84]:
data

,id,num1,num2,num3,num4,num5,target
0,a,950.833174,1,3,-6,-9,0
1,a,1040.526012,9,27,31,4,0
2,a,1170.693920,3,9,3,-6,1
3,a,949.885310,-1,-3,-2,1,0
4,a,1106.604003,2,6,-2,-8,0
...,...,...,...,...,...,...,...
4995,a,910.421817,5,15,17,2,1
4996,a,1001.719720,-1,-3,-1,2,0
4997,a,970.289808,-3,-9,-1,8,0
4998,a,1093.489105,2,6,-1,-7,0


In [85]:
vf = VIFFilterPandas(
    3, 
   'target',
    ['id']
)
vf.remove_mulitocollinear(data, max_iter=50)


             num1  num3  num4  num5  target
0      950.833174     3    -6    -9       0
1     1040.526012    27    31     4       0
2     1170.693920     9     3    -6       1
3      949.885310    -3    -2     1       0
4     1106.604003     6    -2    -8       0
...           ...   ...   ...   ...     ...
4995   910.421817    15    17     2       1
4996  1001.719720    -3    -1     2       0
4997   970.289808    -9    -1     8       0
4998  1093.489105     6    -1    -7       0
4999  1010.408071    -6    -8    -2       0

[5000 rows x 5 columns]
inf 0
             num1  num3  num5  target
0      950.833174     3    -9       0
1     1040.526012    27     4       0
2     1170.693920     9    -6       1
3      949.885310    -3     1       0
4     1106.604003     6    -8       0
...           ...   ...   ...     ...
4995   910.421817    15     2       1
4996  1001.719720    -3     2       0
4997   970.289808    -9     8       0
4998  1093.489105     6    -7       0
4999  1010.408071    -6

/home/aadi/miniconda3/envs/pyspark_env/lib/python3.8/site-packages/statsmodels/stats/outliers_influence.py:195: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/home/aadi/miniconda3/envs/pyspark_env/lib/python3.8/site-packages/statsmodels/stats/outliers_influence.py:195: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


,num5,target,num3,id,num1
0,-9,0,3,a,950.833174
1,4,0,27,a,1040.526012
2,-6,1,9,a,1170.693920
3,1,0,-3,a,949.885310
4,-8,0,6,a,1106.604003
...,...,...,...,...,...
4995,2,1,15,a,910.421817
4996,2,0,-3,a,1001.719720
4997,8,0,-9,a,970.289808
4998,-7,0,6,a,1093.489105


In [230]:
vf.remove_mulitocollinear(dd, max_iter=2)

Index(['num4', 'num1', 'num5', 'num2'], dtype='object')
Index(['num2', 'num4', 'num1'], dtype='object')


DataFrame[num2: bigint, num4: bigint, num1: double]

In [222]:
vifs.max()

inf

# Pandas

In [10]:
import pandas as pd
import time
from statsmodels.stats.outliers_influence import variance_inflation_factor    
from joblib import Parallel, delayed

class VIFFilterPandas:
    def calculate_vif_(self, X:pd.DataFrame, thresh=5.0):
        variables = [X.columns[i] for i in range(X.shape[1])]
        dropped=True
        while dropped:
            dropped=False
            vif = Parallel(n_jobs=-1,verbose=5)(delayed(variance_inflation_factor)(X[variables].values, ix) for ix in range(len(variables)))

            maxloc = vif.index(max(vif))
            if max(vif) > thresh:
                print(time.ctime() + ' dropping \'' + X[variables].columns[maxloc] + '\' at index: ' + str(maxloc))
                variables.pop(maxloc)
                dropped=True

        print('Remaining variables:')
        print([variables])
        return X[[i for i in variables]]

In [87]:
help(variance_inflation_factor)

Help on function variance_inflation_factor in module statsmodels.stats.outliers_influence:

variance_inflation_factor(exog, exog_idx)
    Variance inflation factor, VIF, for one exogenous variable
    
    The variance inflation factor is a measure for the increase of the
    variance of the parameter estimates if an additional variable, given by
    exog_idx is added to the linear regression. It is a measure for
    multicollinearity of the design matrix, exog.
    
    One recommendation is that if VIF is greater than 5, then the explanatory
    variable given by exog_idx is highly collinear with the other explanatory
    variables, and the parameter estimates will have large standard errors
    because of this.
    
    Parameters
    ----------
    exog : {ndarray, DataFrame}
        design matrix with all explanatory variables, as for example used in
        regression
    exog_idx : int
        index of the exogenous variable in the columns of exog
    
    Returns
    -------
  

In [103]:
from functools import partial
import multiprocessing

vif_curried = partial(variance_inflation_factor, exog=data.drop(['id'], axis=1))

pool = multiprocessing.Pool(6)

In [109]:
def vif(exog, exog_idx):
    k_vars = exog.shape[1]

vif(dd.pandas_api(), 1)